In [1]:
# Imports

import numpy as np
import math
import yfinance as yf
import pandas as pd
from datetime import datetime


%run "./Back-End/Pricing Models/Black-Scholes/calc_risk_free_rate.ipynb"

In [2]:
def binomial_tree(S, K, T, r, N, v, option_type):
    """
    Calculate the option price using the Binomial Tree method, with parameters aligned to Black-Scholes.
    
    Parameters:
    S: float - Current stock price
    K: float - Strike price
    T: float - Time to maturity in years
    r: float - Risk-free interest rate
    N: int - Number of steps in the binomial tree
    v: float - Volatility of the underlying asset 
    option_type: str - 'C' for Call or 'P' for Put 
    
    Returns:
    float - Option price
    """
    # Calculate time step and risk-neutral probabilities

    dt = T / N                              # Length of each time step
    u = math.exp(v * math.sqrt(dt))         # Upward movement factor
    d = 1 / u                               # Downward movement factor (ensures no arbitrage)
    p = (math.exp(r * dt) - d) / (u - d)    # Risk-neutral probability
    q = 1 - p                               # Complement probability
    
    # Initialize asset prices at maturity
    asset_prices = S * d**(np.arange(N, -1, -1)) * u**(np.arange(0, N+1, 1))
    
    # Initialize option values at maturity
    if option_type.upper() == 'C':  # Call option
        option_values = np.maximum(asset_prices - K, 0)
    elif option_type.upper() == 'P':  # Put option
        option_values = np.maximum(K - asset_prices, 0)
    else:
        raise ValueError("Invalid option_type. Use 'C' for Call or 'P' for Put.")
    
    # Perform backward induction
    for i in range(N, 0, -1):
        option_values = math.exp(-r * dt) * (p * option_values[1:i+1] + q * option_values[0:i])
    
    return option_values[0]


In [3]:
# Fetch the risk-free rate
r = fetch_risk_free_rate(date_str="2024-01-01")

# Call Option Example
call_price = binomial_tree(S=100, K=100, T=1, r=r, N=3, v=0.2, option_type='C')
print(f"Call Option Price: {call_price:.2f}")

# Put Option Example
put_price = binomial_tree(S=100, K=100, T=1, r=r, N=3, v=0.2, option_type='P')
print(f"Put Option Price: {put_price:.2f}")

Call Option Price: 10.52
Put Option Price: 6.65
